<a href="https://colab.research.google.com/github/Eman-Tedias/Udacity_movie_rental/blob/main/Project_eman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **QUESTION 1**

**SQL CODE:**

WITH c AS
(
       SELECT NAME,
              category_id
       FROM   category), f AS
(
       SELECT title,
              film_id
       FROM   film), r AS
(
       SELECT rental_id,
              inventory_id
       FROM   rental), everything AS
(
       SELECT r.rental_id,
              f.title,
              c.NAME
       FROM   r
       JOIN   inventory
       ON     r.inventory_id = inventory.inventory_id
       JOIN   f
       ON     inventory.film_id = f.film_id
       JOIN   film_category
       ON     f.film_id = film_category.film_id
       JOIN   c
       ON     film_category.category_id = c.category_id)
SELECT   title             AS film_title,
         NAME              AS category_name,
         Count (rental_id) AS rental_count
FROM     everything
GROUP BY 2,
         1
ORDER BY 3 DESC limit 20

**VISUALIZATION CODE:**

In [ ]:
import pandas as pd
import plotly.graph_objects as go
df = pd.read_csv('data/set 1-1 att.csv') #personal directory

fig = go.Figure(data=[go.Table(header=dict(values=['Film Title',
                                                   'Film Category',
                                                   'Rental Count']),
                 cells=dict(values=[df['film_title'],
                                    df['category_name'],
                                    df['rental_count']]))
                     ])
fig.show()

# QUESTION 2

**SQL CODE:**

SELECT Date_part('month', r.rental_date) AS Rental_month,
       Date_part('year', r.rental_date)  AS Rental_year,
       s.store_id,
       Count(r.rental_id)                AS Count_rentals
FROM   store AS s
       JOIN staff AS st
         ON s.store_id = st.store_id
       JOIN payment AS p
         ON st.staff_id = p.staff_id
       JOIN rental AS r
         ON p.rental_id = r.rental_id
GROUP  BY 3,
          1,
          2
ORDER  BY 2,
          1 

**VISUALIZATION CODE:**

In [ ]:
import plotly.express as px

df = pd.read_csv('data/set 2-1 att.csv') #personal directory
fig = px.histogram(df, x='rental_month', y='count_rentals',
             color='store_id', barmode='group',
              title='Rental count from June to August 2005',
               width=750, labels={'rental_month': "Rental Month",
                                  'count_rentals': "Rental Count",
                                  'store_id': "Store ID"})
fig.show()




# **QUESTION 3**

**SQL CODE:**

WITH compound AS
(
       SELECT Date_trunc('month', p.payment_date)          AS pay_mon,
                     Concat(c.first_name,' ', c.last_name) AS fullname,
              p.payment_id                                 AS pay,
              p.amount                                     AS amount
       FROM   payment                                      AS p
       JOIN   customer                                     AS c
       ON     c.customer_id = p.customer_id), person_pay AS
(
         SELECT   fullname,
                  Rank() OVER (ORDER BY Sum(amount) DESC) AS p_rank
         FROM     compound
         GROUP BY 1 limit 10)
SELECT   c.fullname,
         Count(c.pay)  AS pay_countpermon,
         Sum(c.amount) AS pay_amount
FROM     compound      AS c
JOIN     person_pay    AS pp
ON       c.fullname = pp.fullname
GROUP BY 1
ORDER BY 3 DESC

**VISUALIZATION CODE:**

In [ ]:
df = pd.read_csv('data/set 2-2 att.csv') #personal directory
fig = go.Figure(data=[go.Table(header=dict(values=['Customer',
                                                   'Total Rentals',
                                                   'Total Amount']),
                 cells=dict(values=[df['fullname'],
                                    df['pay_countpermon'],
                                    df['pay_amount']]))])
fig.show()

df2 = pd.read_csv('data/set 2-2 att2.csv') #personal directory
fig2 = px.line(df2, x='pay_mon', y='pay_amount', color='fullname',
               markers=True,
               title='Top 10 customers from February to May 2007',
               labels={'pay_mon': 'Month',
                       'pay_amount': 'Pay Amount',
                       'fullname': 'Customer'})
fig2.update_layout(xaxis={'dtick': 1})
fig2.show()

# **QUESTION 4**

**SQL CODE:**

WITH compound AS
(
       SELECT To_char(p.payment_date, 'MM-YYYY')           AS pay_mon,
                     Concat(c.first_name,' ', c.last_name) AS fullname,
              p.payment_id                                 AS pay,
              p.amount                                     AS amount
       FROM   payment                                      AS p
       JOIN   customer                                     AS c
       ON     c.customer_id = p.customer_id), person_pay AS
(
         SELECT   fullname,
                  Rank() OVER (ORDER BY Sum(amount) DESC) AS p_rank
         FROM     compound
         GROUP BY 1 limit 10), pay_rank AS
(
         SELECT   c.pay_mon AS pay_mon,
                  c.fullname,
                  Count(c.pay)  AS pay_countpermon,
                  Sum(c.amount) AS pay_amount
         FROM     compound      AS c
         JOIN     person_pay    AS pp
         ON       c.fullname = pp.fullname
         GROUP BY 2,
                  1
         ORDER BY 2), lag_cols AS
(
         SELECT   pay_mon,
                  fullname,
                  pay_amount                                                       AS cur_mon,
                  Lag(pay_amount, 1) OVER (partition BY fullname ORDER BY pay_mon) AS past_mon
         FROM     pay_rank)
SELECT fullname,
       pay_mon,
       cur_mon,
       past_mon,
       cur_mon - past_mon AS dif
FROM   lag_cols

**VISUALIZATION CODE:**

In [ ]:
import pandas as pd

df = pd.read_csv('data/set 2-3 att.csv').fillna(' ') #personal directory
fig = go.Figure(data=[go.Table(header=dict(values=['Customer',
                                                   'Month',
                                                   'Current Month Amount',
                                                   'Last Month Amount',
                                                   'Difference']),
                 cells=dict(values=[df['fullname'],
                                    df['pay_mon'],
                                    df['cur_mon'],
                                    df['past_mon'],
                                    df['dif']]))])
fig.show()